# TRAINNING A CLASSIFIER

+ Generally while dealing with image, text, audio or video data, standard python packages can be used to load data into numpy array. Then it can be converted to torch.Tensor
    + For __Images__ , packages such as __Pillow__, __OpenCV__ are useful
    + For __audio__, packages suca as __scipy__ and __librosa__
    + For __text__, either raw __Python__ or __Cython__ based loading, or __NLTK__ and __SpaCy__ are useful 
+ For vision, package called __torchvision__ can be used which consists of dataloaders for common datasets such as __Imagenet__, __CIFAR10__, __MNIST__ etc. viz. __torchvision.datasets__ and __torch.utils.data.DataLoader__.
+ LOADING AND NORMALIZING CIFAR10
    + __torchvision.transforms__ can be used to transform the image during loading of the images
    + __transforms.Compose__ can be used to compose transform with functions defined in __transforms__
    + datasets are defined in functions in __torchvision.datasets__ like __torchvision.datasets.CIFAR10()__ with __root__ , __train__, __download__, __transform__.
    + __torch.utils.data.Dataloader__ can be used to load the dataset of required __batch_size__, __shuffle__ specifying __num_workers__ for data_loading
    + __torchvision.utils.makegrid()__ make a grid of batch of image passed and returns the grid image
    + create __iter__ of the __dataloader__ and with .next() function of iterator image and labels are obtained
+ While testing, do under __with torch.no_grad():__
+ __torch.squeeze(input, dim, out)__ : Returns a tensor with all the dimension of input of size 1 removed. For example, if input is of shape: $(A \times 1 \times B \times C \times 1 \times D)$ then the out tensor will be of shape: $(A \times B \times C \times D)$.
+ for dataset to be used in DataLoader, we need to implement __\_\_getitem\_\_()__

### 1. LOADING AND NORMALIZING CIFAR10

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from tensorboardX import SummaryWriter

In [39]:
transform = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
                                ])

trainset = torchvision.datasets.CIFAR10(root='./traindata',
                                        train=True,
                                        transform=transform,
                                       download=True)
trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=4,
                                         shuffle=True,
                                         num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./testdata',
                                      train = False,
                                      download = True,
                                      transform=transform)
testloader = torch.utils.data.DataLoader(testset,
                                        batch_size=4,
                                        shuffle=False,
                                        num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
          'deer', 'dog', 'frog', 'horse',
          'ship', 'truck')
image, label = next(iter(trainloader))
print(image.shape)

Files already downloaded and verified
Files already downloaded and verified
torch.Size([4, 3, 32, 32])


In [3]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np

def imshow(img):
    img = img/2 + 0.5
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)))
    plt.show()

dataiter = iter(trainloader)
images, labels = dataiter.next()
imshow(torchvision.utils.make_grid(images, nrow= 4))
print(' '.join('%5s'%classes[labels[j]] for j in range(4)))

<IPython.core.display.Javascript object>

 frog  bird plane  deer


### 2. Define a Convolutional Neural Network

In [4]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3,6,5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84,10)
    
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
net = Net()


### 3. Define a Loss function and optimizer

In [5]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [6]:
print(list(net.parameters())[0].size())

torch.Size([6, 3, 5, 5])


### 4. Train the network

In [47]:
def normalize(img):
    max = torch.max(img.flatten())
    min = torch.min(img.flatten())
    return (img - min)/(max - min)

In [74]:
writer = SummaryWriter()

for epoch in range(2):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f'%(epoch, i+1, running_loss/100))
            writer.add_scalar('running_loss', running_loss/100, global_step=i+1)
            running_loss = 0.0
            param = list(net.parameters())[0]
            writer.add_image('conv1', torchvision.utils.make_grid(normalize(param).view(-1,1,5,5)), global_step=i+1)
            writer.add_histogram('conv1_grad',param.grad, i+1)
            param = list(net.parameters())[2]
            writer.add_image('conv2', torchvision.utils.make_grid(normalize(param).view(-1,1,5,5)), global_step=i+1, dataformats='CHW')
            writer.add_histogram("conv2_grad", param.grad, i+1)
print('Finished Training')

[0,   100] loss: 1.440
[0,   200] loss: 1.505
[0,   300] loss: 1.321
[0,   400] loss: 1.473
[0,   500] loss: 1.504
[0,   600] loss: 1.331
[0,   700] loss: 1.371
[0,   800] loss: 1.476
[0,   900] loss: 1.397
[0,  1000] loss: 1.373
[0,  1100] loss: 1.468
[0,  1200] loss: 1.391
[0,  1300] loss: 1.345
[0,  1400] loss: 1.415
[0,  1500] loss: 1.392
[0,  1600] loss: 1.332
[0,  1700] loss: 1.390
[0,  1800] loss: 1.313
[0,  1900] loss: 1.388
[0,  2000] loss: 1.326
[0,  2100] loss: 1.340
[0,  2200] loss: 1.341
[0,  2300] loss: 1.428
[0,  2400] loss: 1.374
[0,  2500] loss: 1.446
[0,  2600] loss: 1.372
[0,  2700] loss: 1.430
[0,  2800] loss: 1.374
[0,  2900] loss: 1.316
[0,  3000] loss: 1.413
[0,  3100] loss: 1.429
[0,  3200] loss: 1.396
[0,  3300] loss: 1.349
[0,  3400] loss: 1.372
[0,  3500] loss: 1.393
[0,  3600] loss: 1.401
[0,  3700] loss: 1.406
[0,  3800] loss: 1.267
[0,  3900] loss: 1.311
[0,  4000] loss: 1.470
[0,  4100] loss: 1.460
[0,  4200] loss: 1.396
[0,  4300] loss: 1.306
[0,  4400] 

### 5. Test the network on the test data

In [75]:
dataiter = iter(testloaderaderoader)
images, labels = dataiter.next(torchvisionw(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s'%classes[labels[j]] for j in range(4)))

GroundTruth:    cat  ship  ship plane


In [76]:
outputs = net(images)
_, predicted = torch.max(outputs, 1)
print('Predicted: ',' '.join('%5s' % classes[predictedicted[j]] for j in range(4)))

Predicted:    cat   car   car plane


In [77]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += label.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy of the network on the 10000 test images: %d %%'%(100*correct/total))

Accuracy of the network on the 10000 test images: 58 %


+ __torch.squeeze(input, dim, out)__ : Returns a tensor with all the dimension of input of size 1 removed. For example, if input is of shape: $(A \times 1 \times B \times C \times 1 \times D)$ then the out tensor will be of shape: $(A \times B \times C \times D)$.

In [79]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        print(predicted == labels)
        print((predicted == labels).squeeze())
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1
for i in range(10):
    print('Accuracy of %5s : %2d %%'%(classes[i], 100*class_correct[i]/class_total[i]))

tensor([1, 0, 0, 1], dtype=torch.uint8)
tensor([1, 0, 0, 1], dtype=torch.uint8)
tensor([0, 1, 1, 1], dtype=torch.uint8)
tensor([0, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1], dtype=torch.uint8)
tensor([0, 1, 1, 0], dtype=torch.uint8)
tensor([0, 1, 1, 0], dtype=torch.uint8)
tensor([1, 0, 0, 1], dtype=torch.uint8)
tensor([1, 0, 0, 1], dtype=torch.uint8)
tensor([0, 0, 0, 0], dtype=torch.uint8)
tensor([0, 0, 0, 0], dtype=torch.uint8)
tensor([0, 0, 1, 0], dtype=torch.uint8)
tensor([0, 0, 1, 0], dtype=torch.uint8)
tensor([0, 1, 0, 0], dtype=torch.uint8)
tensor([0, 1, 0, 0], dtype=torch.uint8)
tensor([1, 0, 1, 0], dtype=torch.uint8)
tensor([1, 0, 1, 0], dtype=torch.uint8)
tensor([0, 1, 1, 1], dtype=torch.uint8)
tensor([0, 1, 1, 1], dtype=torch.uint8)
tensor([0, 1, 1, 1], dtype=torch.uint8)
tensor([0, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 0], dtype=torch.uint8)
tensor([1, 1, 1, 0], dtype=torch.uint8)
tensor([1, 1, 1, 1], dtype=torch.uint8)


tensor([0, 1, 0, 0], dtype=torch.uint8)
tensor([0, 1, 0, 0], dtype=torch.uint8)
tensor([1, 0, 0, 1], dtype=torch.uint8)
tensor([1, 0, 0, 1], dtype=torch.uint8)
tensor([0, 0, 1, 1], dtype=torch.uint8)
tensor([0, 0, 1, 1], dtype=torch.uint8)
tensor([0, 1, 1, 1], dtype=torch.uint8)
tensor([0, 1, 1, 1], dtype=torch.uint8)
tensor([1, 0, 0, 1], dtype=torch.uint8)
tensor([1, 0, 0, 1], dtype=torch.uint8)
tensor([0, 1, 0, 1], dtype=torch.uint8)
tensor([0, 1, 0, 1], dtype=torch.uint8)
tensor([1, 0, 1, 1], dtype=torch.uint8)
tensor([1, 0, 1, 1], dtype=torch.uint8)
tensor([0, 1, 0, 0], dtype=torch.uint8)
tensor([0, 1, 0, 0], dtype=torch.uint8)
tensor([1, 1, 0, 1], dtype=torch.uint8)
tensor([1, 1, 0, 1], dtype=torch.uint8)
tensor([1, 1, 0, 0], dtype=torch.uint8)
tensor([1, 1, 0, 0], dtype=torch.uint8)
tensor([1, 1, 0, 1], dtype=torch.uint8)
tensor([1, 1, 0, 1], dtype=torch.uint8)
tensor([1, 1, 0, 1], dtype=torch.uint8)
tensor([1, 1, 0, 1], dtype=torch.uint8)
tensor([1, 0, 0, 0], dtype=torch.uint8)


tensor([1, 1, 0, 1], dtype=torch.uint8)
tensor([1, 1, 0, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1], dtype=torch.uint8)
tensor([0, 0, 1, 1], dtype=torch.uint8)
tensor([0, 0, 1, 1], dtype=torch.uint8)
tensor([1, 0, 0, 1], dtype=torch.uint8)
tensor([1, 0, 0, 1], dtype=torch.uint8)
tensor([1, 1, 0, 1], dtype=torch.uint8)
tensor([1, 1, 0, 1], dtype=torch.uint8)
tensor([1, 1, 0, 1], dtype=torch.uint8)
tensor([1, 1, 0, 1], dtype=torch.uint8)
tensor([0, 0, 0, 1], dtype=torch.uint8)
tensor([0, 0, 0, 1], dtype=torch.uint8)
tensor([1, 0, 0, 1], dtype=torch.uint8)
tensor([1, 0, 0, 1], dtype=torch.uint8)
tensor([0, 1, 0, 1], dtype=torch.uint8)
tensor([0, 1, 0, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1], dtype=torch.uint8)
tensor([0, 0, 1, 0], dtype=torch.uint8)
tensor([0, 0, 1, 0], dtype=torch.uint8)
tensor([0, 1, 0, 1], dtype=torch.uint8)
tensor([0, 1, 0, 1], dtype=torch.uint8)
tensor([0, 1, 0, 1], dtype=torch.uint8)


tensor([0, 0, 1, 0], dtype=torch.uint8)
tensor([0, 0, 1, 0], dtype=torch.uint8)
tensor([0, 1, 1, 0], dtype=torch.uint8)
tensor([0, 1, 1, 0], dtype=torch.uint8)
tensor([1, 0, 1, 1], dtype=torch.uint8)
tensor([1, 0, 1, 1], dtype=torch.uint8)
tensor([1, 0, 1, 1], dtype=torch.uint8)
tensor([1, 0, 1, 1], dtype=torch.uint8)
tensor([0, 0, 0, 1], dtype=torch.uint8)
tensor([0, 0, 0, 1], dtype=torch.uint8)
tensor([1, 0, 1, 0], dtype=torch.uint8)
tensor([1, 0, 1, 0], dtype=torch.uint8)
tensor([0, 1, 0, 1], dtype=torch.uint8)
tensor([0, 1, 0, 1], dtype=torch.uint8)
tensor([0, 1, 0, 0], dtype=torch.uint8)
tensor([0, 1, 0, 0], dtype=torch.uint8)
tensor([0, 1, 1, 1], dtype=torch.uint8)
tensor([0, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 0], dtype=torch.uint8)
tensor([1, 1, 1, 0], dtype=torch.uint8)
tensor([1, 1, 0, 1], dtype=torch.uint8)
tensor([1, 1, 0, 1], dtype=torch.uint8)
tensor([1, 1, 0, 0], dtype=torch.uint8)
tensor([1, 1, 0, 0], dtype=torch.uint8)
tensor([0, 1, 1, 1], dtype=torch.uint8)


tensor([1, 0, 1, 0], dtype=torch.uint8)
tensor([1, 0, 1, 0], dtype=torch.uint8)
tensor([1, 1, 0, 1], dtype=torch.uint8)
tensor([1, 1, 0, 1], dtype=torch.uint8)
tensor([0, 0, 1, 0], dtype=torch.uint8)
tensor([0, 0, 1, 0], dtype=torch.uint8)
tensor([0, 1, 1, 1], dtype=torch.uint8)
tensor([0, 1, 1, 1], dtype=torch.uint8)
tensor([0, 1, 0, 0], dtype=torch.uint8)
tensor([0, 1, 0, 0], dtype=torch.uint8)
tensor([0, 0, 1, 0], dtype=torch.uint8)
tensor([0, 0, 1, 0], dtype=torch.uint8)
tensor([0, 0, 1, 1], dtype=torch.uint8)
tensor([0, 0, 1, 1], dtype=torch.uint8)
tensor([1, 0, 1, 1], dtype=torch.uint8)
tensor([1, 0, 1, 1], dtype=torch.uint8)
tensor([1, 0, 1, 1], dtype=torch.uint8)
tensor([1, 0, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1], dtype=torch.uint8)
tensor([0, 1, 1, 1], dtype=torch.uint8)
tensor([0, 1, 1, 1], dtype=torch.uint8)
tensor([0, 0, 1, 0], dtype=torch.uint8)
tensor([0, 0, 1, 0], dtype=torch.uint8)
tensor([1, 1, 1, 0], dtype=torch.uint8)


...

tensor([1, 1, 0, 1], dtype=torch.uint8)
tensor([1, 1, 0, 1], dtype=torch.uint8)
tensor([0, 0, 1, 1], dtype=torch.uint8)
tensor([0, 0, 1, 1], dtype=torch.uint8)
tensor([0, 1, 0, 0], dtype=torch.uint8)
tensor([0, 1, 0, 0], dtype=torch.uint8)
tensor([0, 0, 0, 1], dtype=torch.uint8)
tensor([0, 0, 0, 1], dtype=torch.uint8)
tensor([1, 0, 1, 1], dtype=torch.uint8)
tensor([1, 0, 1, 1], dtype=torch.uint8)
tensor([1, 1, 0, 0], dtype=torch.uint8)
tensor([1, 1, 0, 0], dtype=torch.uint8)
tensor([0, 1, 1, 0], dtype=torch.uint8)
tensor([0, 1, 1, 0], dtype=torch.uint8)
tensor([1, 1, 0, 0], dtype=torch.uint8)
tensor([1, 1, 0, 0], dtype=torch.uint8)
tensor([0, 1, 1, 0], dtype=torch.uint8)
tensor([0, 1, 1, 0], dtype=torch.uint8)
tensor([1, 0, 0, 1], dtype=torch.uint8)
tensor([1, 0, 0, 1], dtype=torch.uint8)
tensor([0, 1, 1, 0], dtype=torch.uint8)
tensor([0, 1, 1, 0], dtype=torch.uint8)
tensor([1, 1, 0, 1], dtype=torch.uint8)
tensor([1, 1, 0, 1], dtype=torch.uint8)
tensor([0, 0, 1, 0], dtype=torch.uint8)


tensor([0, 0, 1, 0], dtype=torch.uint8)
tensor([0, 0, 1, 0], dtype=torch.uint8)
tensor([0, 1, 0, 1], dtype=torch.uint8)
tensor([0, 1, 0, 1], dtype=torch.uint8)
tensor([1, 1, 0, 1], dtype=torch.uint8)
tensor([1, 1, 0, 1], dtype=torch.uint8)
tensor([0, 0, 1, 1], dtype=torch.uint8)
tensor([0, 0, 1, 1], dtype=torch.uint8)
tensor([1, 0, 0, 1], dtype=torch.uint8)
tensor([1, 0, 0, 1], dtype=torch.uint8)
tensor([0, 0, 0, 1], dtype=torch.uint8)
tensor([0, 0, 0, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1], dtype=torch.uint8)
tensor([0, 1, 1, 0], dtype=torch.uint8)
tensor([0, 1, 1, 0], dtype=torch.uint8)
tensor([1, 0, 0, 1], dtype=torch.uint8)
tensor([1, 0, 0, 1], dtype=torch.uint8)
tensor([1, 0, 0, 1], dtype=torch.uint8)
tensor([1, 0, 0, 1], dtype=torch.uint8)
tensor([1, 0, 1, 0], dtype=torch.uint8)
tensor([1, 0, 1, 0], dtype=torch.uint8)
tensor([0, 0, 0, 1], dtype=torch.uint8)
tensor([0, 0, 0, 1], dtype=torch.uint8)
tensor([0, 1, 0, 1], dtype=torch.uint8)


tensor([0, 1, 1, 1], dtype=torch.uint8)
tensor([0, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1], dtype=torch.uint8)
tensor([0, 0, 1, 0], dtype=torch.uint8)
tensor([0, 0, 1, 0], dtype=torch.uint8)
tensor([1, 1, 0, 1], dtype=torch.uint8)
tensor([1, 1, 0, 1], dtype=torch.uint8)
tensor([0, 1, 0, 0], dtype=torch.uint8)
tensor([0, 1, 0, 0], dtype=torch.uint8)
tensor([1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1], dtype=torch.uint8)
tensor([0, 1, 1, 1], dtype=torch.uint8)
tensor([0, 1, 1, 1], dtype=torch.uint8)
tensor([1, 0, 1, 1], dtype=torch.uint8)
tensor([1, 0, 1, 1], dtype=torch.uint8)
tensor([1, 0, 0, 0], dtype=torch.uint8)
tensor([1, 0, 0, 0], dtype=torch.uint8)
tensor([1, 0, 1, 1], dtype=torch.uint8)
tensor([1, 0, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 0, 1, 1], dtype=torch.uint8)
tensor([1, 0, 1, 1], dtype=torch.uint8)
tensor([0, 0, 1, 1], dtype=torch.uint8)


tensor([1, 0, 0, 1], dtype=torch.uint8)
tensor([1, 0, 0, 1], dtype=torch.uint8)
tensor([1, 0, 0, 0], dtype=torch.uint8)
tensor([1, 0, 0, 0], dtype=torch.uint8)
tensor([1, 1, 0, 0], dtype=torch.uint8)
tensor([1, 1, 0, 0], dtype=torch.uint8)
tensor([0, 1, 0, 1], dtype=torch.uint8)
tensor([0, 1, 0, 1], dtype=torch.uint8)
tensor([1, 0, 1, 1], dtype=torch.uint8)
tensor([1, 0, 1, 1], dtype=torch.uint8)
tensor([1, 0, 0, 1], dtype=torch.uint8)
tensor([1, 0, 0, 1], dtype=torch.uint8)
tensor([0, 1, 1, 1], dtype=torch.uint8)
tensor([0, 1, 1, 1], dtype=torch.uint8)
tensor([0, 0, 0, 1], dtype=torch.uint8)
tensor([0, 0, 0, 1], dtype=torch.uint8)
tensor([0, 1, 0, 1], dtype=torch.uint8)
tensor([0, 1, 0, 1], dtype=torch.uint8)
tensor([1, 0, 0, 1], dtype=torch.uint8)
tensor([1, 0, 0, 1], dtype=torch.uint8)
tensor([0, 1, 1, 1], dtype=torch.uint8)
tensor([0, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 0, 0], dtype=torch.uint8)


tensor([0, 0, 1, 0], dtype=torch.uint8)
tensor([0, 0, 1, 0], dtype=torch.uint8)
tensor([1, 0, 0, 1], dtype=torch.uint8)
tensor([1, 0, 0, 1], dtype=torch.uint8)
tensor([1, 0, 1, 1], dtype=torch.uint8)
tensor([1, 0, 1, 1], dtype=torch.uint8)
tensor([1, 0, 1, 1], dtype=torch.uint8)
tensor([1, 0, 1, 1], dtype=torch.uint8)
tensor([1, 0, 1, 1], dtype=torch.uint8)
tensor([1, 0, 1, 1], dtype=torch.uint8)
tensor([0, 1, 1, 1], dtype=torch.uint8)
tensor([0, 1, 1, 1], dtype=torch.uint8)
tensor([0, 0, 1, 0], dtype=torch.uint8)
tensor([0, 0, 1, 0], dtype=torch.uint8)
tensor([0, 1, 1, 0], dtype=torch.uint8)
tensor([0, 1, 1, 0], dtype=torch.uint8)
tensor([1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 0, 1, 0], dtype=torch.uint8)
tensor([1, 0, 1, 0], dtype=torch.uint8)
tensor([0, 1, 0, 1], dtype=torch.uint8)
tensor([0, 1, 0, 1], dtype=torch.uint8)
tensor([0, 0, 0, 0], dtype=torch.uint8)
tensor([0, 0, 0, 0], dtype=torch.uint8)
tensor([0, 0, 0, 1], dtype=torch.uint8)
